### [337\. House Robber III](https://leetcode.com/problems/house-robber-iii/)

Difficulty: **Medium**  

Related Topics: [Tree](https://leetcode.com/tag/tree/), [Depth-first Search](https://leetcode.com/tag/depth-first-search/)


The thief has found himself a new place for his thievery again. There is only one entrance to this area, called the "root." Besides the root, each house has one and only one parent house. After a tour, the smart thief realized that "all houses in this place forms a binary tree". It will automatically contact the police if two directly-linked houses were broken into on the same night.

如果碰到直接相連的節點, 就會觸發警報

尋找最大值

Determine the maximum amount of money the thief can rob tonight without alerting the police.

**Example 1:**

```
Input: [3,2,3,null,3,null,1]

     3
    / \
   2   3
    \   \ 
     3   1

Output: 7 
Explanation: Maximum amount of money the thief can rob = 3 + 3 + 1 = 7.
```

**Example 2:**

```
Input: [3,4,5,1,3,null,1]

     3
    / \
   4   5
  / \   \ 
 1   3   1

Output: 9
Explanation: Maximum amount of money the thief can rob = 4 + 5 = 9.
```

### Solution 1: Failed

DFS Preorder 遍歷二元樹, 之後照著 198. House Robber 的作法

Failed: 因為樹的形狀, 產生的 preorder 陣列不一定能反應層級的關係


In [ ]:
class Solution:
    def rob(self, root: TreeNode) -> int:
        def preOrder(root):
            res, stack = [], [root]
            while stack:
                node = stack.pop()
                if node:
                    stack.append(node.right)
                    stack.append(node.left)
                    res.append(node.val)
            return res
        
        nums = preOrder(root)
        
        if len(nums) == 0:
            return 0
        elif len(nums) <= 2:
            return max(nums)
        elif len(nums) == 3:
            return max(nums[0], nums[1] + nums[2])
        
        pre1 = pre2 = 0
        
        for num in nums:
            temp = pre1
            pre1 = max(pre2 + num, pre1)
            pre2 = temp
        
        return pre1

### [循序漸進](https://leetcode.com/problems/house-robber-iii/discuss/79330/Step-by-step-tackling-of-the-problem)

#### 1. 遞迴 Subtree

```rob(root)```會回傳以 root 為起點最大的結果, 而```rob(root)```又可以從```rob(root.left)```及```rob(root.right)```的結果中得到

遞迴

1. base case: 當 root 為空即可停止
2. 遞迴關係: 兩種狀況, root 有被搶或沒有。如果 root 被搶了那接下來可行的點就是 4 個孫輩的 subtrees (```root.right.right, root.right.left, root.left.right, root.left.left```)。如果 root 沒有被
搶, 接下來就是 2 個子輩的 subtrees (```root.right, root.left```)

time complexity: $O(n^2)$

TLE

In [ ]:
class Solution:
    def rob(self, root: TreeNode) -> int:
        if not root:
            return 0
        val = 0
        if root.left:
            val += self.rob(root.left.left) + self.rob(root.left.right)
        if root.right:
            val += self.rob(root.right.left) + self.rob(root.right.right)
        
        return max(val + root.val, rob(root.left) + rob(root.right))

2. 紀錄重複計算

單純遞迴會有許多重複計算, 只要用記錄下來, 就能實現 DP

所謂 DP 就是: optimal substructure + overlapping of subproblems

time complexity: O(n)

58.56% 

In [ ]:
class Solution:
    def rob(self, root: TreeNode) -> int:
        def robSub(root, lookup):
            if not root:
                return 0
            if root in lookup:
                return lookup[root]
            val = 0

            if root.left:
                val += robSub(root.left.left, lookup) + robSub(root.left.right, lookup)
            
            if root.right:
                val += robSub(root.right.left, lookup) + robSub(root.right.right, lookup)
            
            val = max(val + root.val, robSub(root.left) + robSub(root.right))

            lookup[root] = val

            return val

        
        return robSub(root, {})

3. 退後一步

為何會有重疊的 subproblem: 因為遞迴的過程中 ```rob(root)``` 的狀態沒有被記錄下來

如果可以在一開始就把 root 有沒有被搶過的狀態記錄下來, 就可以避免產生重複的 subproblem

只需要兩個變數: now: 如果這個 root 有被搶; later: 如果這個 root 沒有被搶

In [ ]:
class Solution:
    def rob(self, root: TreeNode) -> int:
        
        def robSub(root):
            if not root:
                return 0
            
            left, right = robSub(root.left), robSub(root.right)

            now = root.val + left[1] + right[1]
            later = max(left) + max(right)

            return (now, later)
        
        return max(robSub(root))
        

In [ ]:
class Solution:
    def rob(self, root: TreeNode) -> int:
        def dfs(node):
            if not node:
                return [0, 0]
            left, right = dfs(node.left), dfs(node.right)
            return [max(left) + max(right), node.val + left[0] + right[0]]
        return max(dfs(root))